In [1]:
!pip install transformers

## Set up

Lets start by importing all of our dependenies and specifying a feww key directories.

In [2]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import pandas as pd
from google.colab import drive
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Mount drive
drive.mount('/content/drive')

# Path to dataset csv
csv_file_path = '/content/drive/My Drive/trained_models/squad_data.csv'
df = pd.read_csv(csv_file_path)

# Set up device to GPU if present and CPU if not
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, lest load our model and tokenizer from the weights we saved from our fine-tuned model.

In [3]:
# load in model and tokenizer
saved_model_path = '/content/drive/My Drive/trained_models/trained_model_bert_csv'
tokenizer = BertTokenizer.from_pretrained(saved_model_path)
model = BertForQuestionAnswering.from_pretrained(saved_model_path)
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

Now, we need to define a function to assist in finding the models most optimal output to the users prompt.

In [4]:
def find_best_answer(query, df):
    # Check cache first
    if query in cache:
        return cache[query]

    # Initialize variables to store the best answer and its score
    best_answer = None
    best_score = float('-inf')

    for idx, row in df.iterrows():
        context = row['Context']

        # Tokenize input
        inputs = tokenizer(query, context, return_tensors='pt', max_length=512, padding=True, truncation=True, add_special_tokens=True)

        # Get model output
        with torch.no_grad():
            outputs = model(**inputs.to(device))

        # Get start and end scores
        start_scores, end_scores = outputs.start_logits, outputs.end_logits

        # Get the best start and end positions for this context
        start_idx = torch.argmax(start_scores)
        end_idx = torch.argmax(end_scores)

        # Calculate the total score for this answer
        score = start_scores[0, start_idx].item() + end_scores[0, end_idx].item()

        # Update the best answer if this answer is better
        if score > best_score:
          best_score = score
          best_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx+1]))

        # Implement early stopping based on a confidence threshold
        if best_score > 0.9:  # Assuming 0.9 is the confidence threshold
            break

        #print(f"Score: {best_score}, Best_answer: {best_answer}")

    # Store the result in cache
    cache[query] = best_answer

    return best_answer


Now, we will define the interaction loop for our model. After this is run, a text input field will pop up below the cell. Enter your prompt into that cell and hit enter to ask the bot. Use the following commands to facillitate your interactions.

* Use "quit" to exit interaction with the bot
* Type "set context to" followed by a subject to switch the bots context to another subject area. Some examples are "Tibet", "Chopin", and "Beyonce".

In [5]:
cache = {}
current_context = ""

while True:
    user_query = input("You: ")
    if user_query.lower() == "quit":
        print("Bot: Goodbye!")
        break

    # Set the context if the user specifies it
    if "set context to" in user_query.lower():
        current_context = user_query.lower().replace("set context to", "").strip()
        print(f"Context set to: {current_context}")
        continue

    # Filter DataFrame based on the current context
    if current_context != None:
        filtered_df = df.reset_index()
        filtered_df = filtered_df.loc[filtered_df['Context'].str.contains(current_context, case=False)]
        if filtered_df.empty:
            print("Bot: Sorry, I don't have information on that context.")
            continue
    else:
        filtered_df = df.reset_index()

    # Find the best answer using BERT
    best_answer = find_best_answer(user_query, filtered_df)
    print(f"Bot: {best_answer}")
    #context = ''


You: Who was in charge?
Bot: mathew knowles
You: set context to tibet
Context set to: tibet
You: Who was in charge
Bot: drogon chogyal phagpa
You: quit
Bot: Goodbye!
